
## Overview

This notebook will show you how to create and query a table or DataFrame that you uploaded to DBFS. [DBFS](https://docs.databricks.com/user-guide/dbfs-databricks-file-system.html) is a Databricks File System that allows you to store data for querying inside of Databricks. This notebook assumes that you have a file already inside of DBFS that you would like to read from.

This notebook is written in **Python** so the default cell type is Python. However, you can use different languages by using the `%LANGUAGE` syntax. Python, Scala, SQL, and R are all supported.

In [0]:
# File location and type
file_location = "/FileStore/tables/study_performance-2.csv"
file_type = "csv"

# CSV options
infer_schema = "false"
first_row_is_header = "true"
delimiter = ","

# The applied options are for CSV files. For other file types, these will be ignored.
df_perf_study = spark.read.format(file_type) \
  .option("inferSchema", infer_schema) \
  .option("header", first_row_is_header) \
  .option("sep", delimiter) \
  .load(file_location)

display(df_perf_study)

gender,race_ethnicity,parental_level_of_education,lunch,test_preparation_course,math_score,reading_score,writing_score
female,group B,bachelor's degree,standard,none,72,72,74
female,group C,some college,standard,completed,69,90,88
female,group B,master's degree,standard,none,90,95,93
male,group A,associate's degree,free/reduced,none,47,57,44
male,group C,some college,standard,none,76,78,75
female,group B,associate's degree,standard,none,71,83,78
female,group B,some college,standard,completed,88,95,92
male,group B,some college,free/reduced,none,40,43,39
male,group D,high school,free/reduced,completed,64,64,67
female,group B,high school,free/reduced,none,38,60,50


In [0]:
%sql
DROP SCHEMA bronze CASCADE

In [0]:
%sql
CREATE DATABASE IF NOT EXISTS bronze

In [0]:
%sql
DROP TABLE IF EXISTS bronze.Study_performance

In [0]:
dbutils.fs.rm('dbfs:/user/hive/warehouse/bronze.db/Study_performance',True)
 
df_perf_study.write.format("delta").mode("append").saveAsTable("bronze.Study_performance")
 
print("CARGA DADOS DE PERFORMANCE DE ESTUDANTES CRIADO COM SUCESSO NA DATABASE BRONZE!")

CARGA DADOS DE PERFORMANCE DE ESTUDANTES CRIADO COM SUCESSO NA DATABASE BRONZE!


In [0]:
## carregando os dados do DATABASE bronze para tratamento dos dados de performance dos estudantes
 
df_perf_study_bronze_sql = spark.sql('''select * from bronze.Study_performance''')

In [0]:
from pyspark.sql.functions import col

df_perf_study_silver = df_perf_study_bronze_sql.withColumn("math_score", col("math_score").cast("int")) \
                     .withColumn("reading_score", col("reading_score").cast("int")) \
                     .withColumn("writing_score", col("writing_score").cast("int"))

In [0]:
%sql
DROP SCHEMA silver CASCADE

In [0]:
%sql
CREATE DATABASE IF NOT EXISTS silver

In [0]:
%sql
DROP TABLE IF EXISTS silver.Study_performance

In [0]:
dbutils.fs.rm('dbfs:/user/hive/warehouse/silver.db/Study_performance',True)
 
df_perf_study_silver.write.format("delta").mode("append").saveAsTable("silver.Study_performance")
 
print("CARGA DADOS DE PERFORMANCE DE ESTUDANTES CRIADO COM SUCESSO NA DATABASE SILVER!")

CARGA DADOS DE PERFORMANCE DE ESTUDANTES CRIADO COM SUCESSO NA DATABASE SILVER!


In [0]:
%sql
DROP SCHEMA gold CASCADE

In [0]:
%sql
CREATE DATABASE IF NOT EXISTS gold

In [0]:
%sql
DROP TABLE IF EXISTS gold.Study_performance

In [0]:
df_perf_study_gold_sql = spark.sql('''select * from silver.Study_performance''')

In [0]:
dbutils.fs.rm('dbfs:/user/hive/warehouse/gold.db/Study_performance',True)
 
df_perf_study_gold_sql.write.format("delta").mode("append").saveAsTable("gold.Study_performance")
 
print("CARGA DADOS DE PERFORMANCE DE ESTUDANTES CRIADO COM SUCESSO NA DATABASE GOLD!")

CARGA DADOS DE PERFORMANCE DE ESTUDANTES CRIADO COM SUCESSO NA DATABASE GOLD!


In [0]:
from pyspark.sql.functions import count

# Filtragem e contagem dos alunos que tiraram nota acima de 80 em Matemática e tiveram almoço padrão
num_students_gold = df_perf_study_gold_sql.filter((col("math_score") > 80) & (col("lunch") == "standard")) \
                             .agg(count("*").alias("num_students"))

# Exibição do resultado
num_students_gold.show()

+------------+
|num_students|
+------------+
|         156|
+------------+



In [0]:
from pyspark.sql.functions import col

# Filtrar alunos com nota de Matemática acima de 80 e almoço padrão
students_filtered = df_perf_study_gold_sql.filter((col('math_score') > 80) & (col('lunch') == 'standard'))


In [0]:
# Contar o número de alunos que satisfazem os critérios
total_students = df_perf_study_gold_sql.count()
students_above_80 = students_filtered.count()

# Calcular o percentual
percent_above_80 = (students_above_80 / total_students) * 100

# Arredondar para duas casas decimais
percent_above_80 = round(percent_above_80, 2)

print(f"Percentual de alunos com nota acima de 80 em Matemática e almoço padrão: {percent_above_80}%")


Percentual de alunos com nota acima de 80 em Matemática e almoço padrão: 15.6%
